In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (1000 Biscayne Blvd #5401, Miami, FL 33132, US...
2     (10295 Collins Ave UNIT 808, Bal Harbour, FL 3...
3     (18555 Collins Ave APT 2003, Sunny Isles Beach...
4     (2821 S Bayshore Dr #9d, Miami, FL 33133, USA,...
5     (801 South Pointe Dr APT 206, Miami Beach, FL ...
6     (1445 16th St APT 503, Miami Beach, FL 33139, ...
7     (Santa Maria, 1643 Brickell Ave APT 2105, Miam...
8     (19111 Collins Ave APT 3104, Sunny Isles Beach...
9     (3801 Collins Ave #501, Miami Beach, FL 33140,...
10    (2600 Island Blvd, unit #CU-1, 2600 E Island B...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,One Thousand Museum\r1000 Biscayne Blvd 5401\r...,12/15/2022,598,"$18,250,000.00",9200,"$1,983.70",Lourdes Alatriste,Douglas Elliman,Lourdes Alatriste,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd 5401 Miami,One Thousand Museum,1000 Biscayne Blvd 5401 Miami,"(1000 Biscayne Blvd #5401, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
2,One Bal Harbour\r10295 Collins Ave 808\rBal Ha...,12/13/2022,253,"$5,750,000.00",2763,"$2,081.07",Daniel Alitenssi,Prime Int'l Associates Inc,Elizabeth Rodriguez,The Keyes Company,One Bal Harbour 10295 Collins Ave 808 Bal Harbour,One Bal Harbour,10295 Collins Ave 808 Bal Harbour,"(10295 Collins Ave UNIT 808, Bal Harbour, FL 3...","(25.8991765, -80.12308530000001, 0.0)",25.899176,-80.123085,0.0
3,Porsche Design TOwer\r18555 Collins Ave 2003\r...,12/12/2022,328,"$4,500,000.00",3171,"$1,419.11",Luiza Weisberg,London Foster Realty,Max Farbstein,Douglas Elliman,Porsche Design TOwer 18555 Collins Ave 2003 Su...,Porsche Design TOwer,18555 Collins Ave 2003 Sunny Isles Beach,"(18555 Collins Ave APT 2003, Sunny Isles Beach...","(25.9481553, -80.1200734, 0.0)",25.948155,-80.120073,0.0
4,Park Grove Tower II\r2821 S Bayshore Dr 9D\rMiami,12/16/2022,72,"$4,200,000.00",2558,"$1,641.91",Sandra Magalhaes,KB Realty,Carolyn Casey,London Foster Realty,Park Grove Tower II 2821 S Bayshore Dr 9D Miami,Park Grove Tower II,2821 S Bayshore Dr 9D Miami,"(2821 S Bayshore Dr #9d, Miami, FL 33133, USA,...","(25.7287205, -80.23808009999999, 0.0)",25.728721,-80.238080,0.0
5,Marea Condo\r801 S Pointe Dr 206\rMiami Beach,12/15/2022,270,"$3,750,000.00",4732,"$1,480.46",Matias Alem,"BRG International , LLC",Lisa Van Wagenen,Brown Harris Stevens,Marea Condo 801 S Pointe Dr 206 Miami Beach,Marea Condo,801 S Pointe Dr 206 Miami Beach,"(801 South Pointe Dr APT 206, Miami Beach, FL ...","(25.7686007, -80.1361615, 0.0)",25.768601,-80.136162,0.0
6,Capri South Beach\r1445 16th St 503\rMiami Beach,12/14/2022,126,"$2,200,000.00",1919,"$1,146.43",Isabel Dohse,"Compass Florida, LLC",Joanna Jimenez,"Compass Florida, LLC",Capri South Beach 1445 16th St 503 Miami Beach,Capri South Beach,1445 16th St 503 Miami Beach,"(1445 16th St APT 503, Miami Beach, FL 33139, ...","(25.7891908, -80.14467929999999, 0.0)",25.789191,-80.144679,0.0
7,Santa Maria\r1643 Brickell Ave 2105\rMiami,12/13/2022,11,"$2,150,000.00",2580,$833.33,Fabian Dominguez,Fortune International Realty,Maria Velez,Brown Harris Stevens,Santa Maria 1643 Brickell Ave 2105 Miami,Santa Maria,1643 Brickell Ave 2105 Miami,"(Santa Maria, 1643 Brickell Ave APT 2105, Miam...","(25.755259, -80.1947467, 0.0)",25.755259,-80.194747,0.0
8,Ocean Two Condo\r19111 Collins Ave 3104\rSunny...,12/15/2022,198,"$1,900,000.00",2440,$778.69,Lana Shvartsburd,Avanti Way Realty,Miral Amar,One Sotheby's Int'l Realty,Ocean Two Condo 19111 Collins Ave 3104 Sunny I...,Ocean Two Condo,19111 Collins Ave 3104 Sunny Isles Beach,"(19111 Collins Ave APT 3104, Sunny Isles Beach...","(25.9534131, -80.1197571, 0.0)",25.953413,-80.119757,0.0
9,Mosaic\r3801 Collins Ave 501\rMiami Beach,12/12/2022,178,"$1,800,000.00",1824,$986.84,Menachem Fellig,"Compass Florida, LLC",Menachem Fellig,"Compass Florida, LLC",Mosaic 3801 Collins Ave 501 Miami Beach,Mosaic,3801 Collins Ave 501 Miami Beach,"(3801 Collins Ave #501, Miami Beach, FL 33140,...","(25.8113373, -80.12262849999999, 0.0)",25.811337,-80.122628,0.0
10,0Williams Island\r2600 Island Blvd 1802\rAventura,12/12/2022,208,"$1,700,000.00",3200,$531.25,Joel Matus,"Williams Island Realty, LLC",Aaron Gurland,"Syncopated Real Estate, LLC",0Williams Island 2600 Island Blvd 1802 Aventura,,0Williams Island 2600 Island Blvd 1802 Aventura,"(2600 Island Blvd, unit #CU-1, 2600 E Island B...","(25.9392382, -80.1337736, 0.0)",25.939238,-80.133774,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.at[10,'building_name']=('Williams Island')
df.at[10,'address_only']=('2600 Island Blvd unit CU-1')

In [17]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [11]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('December 11th - December 17th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [21]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [12]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [13]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_121722
